In [3]:
from pyspark.sql import SparkSession
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.pretrained import *

from pyspark.sql import SparkSession
from pyspark.context import SparkContext 
import pyspark.sql.functions as F
from pyspark import SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
import os 
os.environ['PYSPARK_PYTHON'] = "/root/miniconda3/envs/sp/bin/python"

spark = SparkSession.builder\
  .config(
    "spark.jars", 
    "/data/jupyter-data/lab08/kafka-clients-3.4.0.jar,/data/jupyter-data/lab08/spark-sql-kafka-0-10_2.12-3.3.1.jar, \
    /data/jupyter-data/lab08/spark-token-provider-kafka-0-10_2.12-3.3.1.jar, \
    /data/jupyter-data/lab08/commons-pool2-2.11.1.jar") \
  .config("spark.executorEnv.PYSPARK_PYTHON","/root/miniconda3/envs/sp/bin/python") \
  .config("spark.executor.memory", "2g") \
  .config("spark.driver.memory", "2g") \
  .appName("lab9_exercise").getOrCreate()

In [4]:
from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')


##  分割线

In [5]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, StructType, StructField

# 定义UDF，将每个question和context传递给question_answerer模型，并返回答案
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')
question_answerer_udf = udf(lambda context, question: (question_answerer(question=question, context=context)['answer'], question_answerer(question=question, context=context)['score']), StructType([StructField('answer', StringType()), StructField('score', DoubleType())]))

# 定义输入流，schema包含context和question两列
input_schema = StructType([
    StructField("context", StringType()),
    StructField("question", StringType()),
     StructField("answer", StringType())
])
stream = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "q3") \
    .option("startingOffsets", "latest") \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", input_schema).alias("input")) \
    .select("input.*")




In [6]:
query = stream.writeStream \
    .outputMode("append") \
    .format("memory") \
    .option("queryName", "QA_model") \
    .trigger(processingTime = '5 seconds')
    
query.start()

23/05/16 08:21:17 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d15b0db8-3c1b-4752-a786-1c35dec6d438. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/16 08:21:17 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [7]:
# 定义处理批次的函数
def process_batch(df, epoch_id):
    # 将结果写入到指定的存储位置，例如一个parquet文件
    result_df = df.withColumn("result", question_answerer_udf(col("context"), col("question")))
    result_df = result_df.withColumn("model_answer", col("result").getItem("answer")).withColumn("score", col("result").getItem("score")).drop("result")
    result_df.select("question","answer","model_answer","score").show(truncate=False)
    result_df.select("question","answer","model_answer","score").write.mode("append").parquet("output/answers.parquet")
# 启动流式处理，并将数据分批处理
stream.writeStream.foreachBatch(process_batch).start().awaitTermination()

23/05/16 08:21:27 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3db3e89a-b2e2-4796-88ef-e52882c7d9a7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/16 08:21:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
+--------+------+------------+-----+
|question|answer|model_answer|score|
+--------+------+------------+-----+
+--------+------+------------+-----+



23/05/16 08:21:30 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894


23/05/16 08:21:31 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/05/16 08:21:31 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/05/16 08:21:31 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894


2023-05-16 08:21:31.639827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 08:21:31.770610: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-16 08:21:31.774423: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-16 08:21:31.774436: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

+----------------------------------------------------------------------------------------------------------+--------------------------+--------------------------+-------------------+
|question                                                                                                  |answer                    |model_answer              |score              |
+----------------------------------------------------------------------------------------------------------+--------------------------+--------------------------+-------------------+
|The defeat of the Arab troops in the Six-Day War constituted what for the Arab Muslim world?              |pivotal event             |pivotal event             |0.47990700602531433|
|Secular Arab nationalism was blamed for both the defeat of Arab troops as well as what type of stagnation?|economic                  |secular                   |0.5412832498550415 |
|Secular Arab nationalism was blamed for both the defeat of Arab troops as well as wh

...

+-------------------------------------------------------------+---------------+------------+------------------+
|question                                                     |answer         |model_answer|score             |
+-------------------------------------------------------------+---------------+------------+------------------+
|What is the kilogram-force sometimes reffered to as?         |kilopond       |kilopond    |0.748073935508728 |
|What is a very seldom used unit of mass in the metric system?|slug           |metric slug |0.3782065510749817|
|What is a very seldom used unit of mass in the metric system?|metric slug    |metric slug |0.3782065510749817|
|What is a very seldom used unit of mass in the metric system?|the metric slug|metric slug |0.3782065510749817|
+-------------------------------------------------------------+---------------+------------+------------------+

23/05/16 08:36:38 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It

23/05/16 08:36:39 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/05/16 08:36:39 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894


+------------------------------------------------------------------------+------+------------+-------------------+
|question                                                                |answer|model_answer|score              |
+------------------------------------------------------------------------+------+------------+-------------------+
|What seldom used term of a unit of force equal to 1000 pound s of force?|kip   |the kip     |0.14559562504291534|
|What is the seldom used force unit equal to one thousand newtons?       |sthène|metric slug |0.04813629388809204|
+------------------------------------------------------------------------+------+------------+-------------------+

23/05/16 08:36:40 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/05/16 08:36:40 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDa

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/root/miniconda3/envs/sp/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/root/miniconda3/envs/sp/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/root/miniconda3/envs/sp/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [8]:
processed_data = spark.read.parquet("output/answers.parquet")
processed_data.show()

+--------------------+--------------------+--------------------+-------------------+
|            question|              answer|        model_answer|              score|
+--------------------+--------------------+--------------------+-------------------+
|What happened to ...|steep and steady ...|A steep and stead...| 0.4914613366127014|
|What gained groun...|anti-democratic I...|different democra...| 0.4869297444820404|
|What gained groun...|anti-democratic I...|different democra...| 0.4869297444820404|
|Who was the ideol...|        Ali Shariati|        Ali Shariati| 0.9576330184936523|
|Mohammad Iqbal wa...|         ideological|         ideological| 0.9344263672828674|
|Where does Khomei...|   somewhere between|   somewhere between|0.41264471411705017|
|Where does Khomei...|             between|   somewhere between|0.41264471411705017|
|Who was it essent...|the Prophet Mohammad|    Prophet Mohammad|0.43677428364753723|
|Who was it essent...|Prophet Mohammad ...|    Prophet Mohammad|0

In [ ]:
# 定义处理批次的函数
def process_batch(df, epoch_id):
    # 使用question_answerer模型对每个question和context进行问答
    # 输出结果到控制台 
    result_df = df.withColumn("result", question_answerer_udf(col("context"), col("question")))
    result_df = result_df.withColumn("answer1", col("result").getItem("answer")).withColumn("score", col("result").getItem("score")).drop("result")
    result_df.select("question","answer","answer1","score").show(truncate=False)
# 启动流式处理，并将数据分批处理
stream.writeStream.foreachBatch(process_batch).start().awaitTermination()

In [ ]:
processed_data = spark.read.parquet("output/answers.parquet")
processed_data.show()

In [12]:
schema = StructType([
    StructField("question", StringType(), True),
    StructField("context", StringType(), True),
    StructField("answer", StringType(), True)
    
])
stream = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "q3") \
    .option("startingOffsets", "earliest") \
    .load()
parsedStream = stream.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

In [9]:
parsedStream

DataFrame[question: string, context: string, answer: string]

In [11]:
resultStream = parsedStream.selectExpr("question", "context") \
    .rdd \
    .map(lambda x: (x.question, x.context, question_answerer(question=x.question, context=x.context))) \
    .toDF(["question", "context", "answer"]) \
    .selectExpr("question", "context", "answer['answer'] AS answer", "answer['score'] AS score", "answer['start'] AS start", "answer['end'] AS end")

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
kafka